In [1]:
import json
import os
import sys
import geopandas as gpd
import pandas as pd
from apsim.apsim_output_parser import parse_summary_output_field
import analyses.munging as munge

In [7]:
#biocentury_fields = ['ebilsland', 'kfinch', 'accola']
biocentury_fields = ['nbilsland', 'sbilsland']
#strips_fields = ['basswood', 'interim', 'orbweavernorth', 'orbweaversouth']

In [8]:
# field = json.loads( open( f'field_processing_jsons/ebilsland.json', 'r' ).read() )
# print(json.dumps(field, indent=1))

In [16]:
def make_outdir_folders(field_list):
    for i in field_list:
        field_folder = os.path.join('out_files', i, 'shape_files')
        if not os.path.exists(field_folder):
            os.makedirs(field_folder)
make_outdir_folders(biocentury_fields)

In [10]:
def create_sampled_shapefile(field_list):
    for i in field_list:
        field = json.loads( open( f'field_processing_jsons/{i}.json', 'r' ).read() )
        field_folder = os.path.join('out_files', i, 'shape_files')
        # if not os.path.exists(field_folder):
        #     os.mkdirs(field_folder)
        for i in field['year']:
            field_name = field['field_name']
            twi_path = field['twi_file']
            ssurgo_path = field['ssurgo_file']
            ym_file = field['year'][i]['ym_file']
            rotation = field['year'][i]['rotation']
            crop = field['year'][i]['crop']
            apsim_folder = field['apsim_files']
            met_file = field['met_file']
            june_ndvi_date = f'{i}06'
            july_ndvi_date = f'{i}07'
            june_ndvi_file = ''
            july_ndvi_file = ''
            #set target crs from gpd df
            ym_gdf = gpd.read_file(ym_file)
            target_crs = ym_gdf.crs
            #search for june and july ndvi files
            for file in os.listdir(f'ndvi/{field_name}'):
                if june_ndvi_date in file:
                    june_ndvi_file = file
                if july_ndvi_date in file:
                    july_ndvi_file = file
            if june_ndvi_file == '':
                print(f'June NDVI file not found for {field_name} {i}.')
                continue
            if july_ndvi_file == '':
                print(f'July NDVI file not found for {field_name} {i}.')
                continue
            #sample twi to ym file
            twi_out_path = os.path.join('out_files', field_name, f'{field_name}_twi_26915.tif')
            twi_gdf = munge.prepare_twi_df(ym_file, twi_path, twi_out_path, target_crs, i, field_name, crop)
            ## sample ndvi to new twi_gdf
            #first get appropirate in and out paths
            jun_ndvi_in_path = os.path.join('ndvi', field_name, june_ndvi_file)
            jun_ndvi_out_path = os.path.join('out_files', field_name, f'{field_name}_ndvi_twi_jun_{i}_26915.tif')
            jul_ndvi_in_path = os.path.join('ndvi', field_name, july_ndvi_file)
            jul_ndvi_out_path = os.path.join('out_files', field_name, f'{field_name}_ndvi_twi_jul_{i}_26915.tif')
            #then sample for june and for july
            jun_ndvi_twi_gdf = munge.prepare_ndvi_df(twi_gdf, 'jun', jun_ndvi_in_path, jun_ndvi_out_path, target_crs, all_touched=False)
            jul_ndvi_twi_gdf = munge.prepare_ndvi_df(jun_ndvi_twi_gdf, 'jul', jul_ndvi_in_path, jul_ndvi_out_path, target_crs, all_touched=False)
            ## met
            met_path = os.path.join('met_files', 'nasapwr', f'{met_file}')
            ndvi_twi_met_gdf = munge.prepare_met_df(met_path, jul_ndvi_twi_gdf, i, header=7, precip_col='rain')
            ## ssurgo
            ssurgo_out_path = os.path.join('out_files', field_name, f'{field_name}_ssurgo_26915.geojson')
            ndvi_twi_met_ssurgo_gdf = munge.prepare_ssurgo_df(ndvi_twi_met_gdf, ssurgo_path, ssurgo_out_path, target_crs)
            ## apsim files
            apsim_files_path = os.path.join('apsim_files', apsim_folder, i, rotation)
            write_file = f'{field_name}_{crop}_{i}_full'
            munge.prepare_apsim_full_df(ndvi_twi_met_ssurgo_gdf, apsim_files_path, int(i), field_folder, write_file)

In [12]:
create_sampled_shapefile(biocentury_fields)

RasterioIOError: Attempt to create new tiff file 'out_files\nbilsland\nbilsland_twi_26915.tif' failed: No such file or directory

In [ ]:
# year = '2016'
# precip_col = 'rain'
# data_type_dict = {
#     'year' : str,
#     'day' : int,
#     'radn' : float,
#     'maxt' : float,
#     'mint' : float,
#     'rain' : float
# }
# met_df = pd.read_csv('met_files/nasapwr/orbweaversouth-1990-2020.met', header=7, sep=' ')
# met_df = met_df.drop([0])
# met_df = met_df.astype( data_type_dict )
# met_df = met_df.loc[met_df['year'] == '2016']
# top10_precip_events_df = munge.get_top_ten_days(met_df, str(year), 'year', precip_col)
# top10_precip_events_df
# adjacent_days_list = munge.check_adjacent_days(top10_precip_events_df, 'day', precip_col)


# # #224, 225
# #for i in adjacent_days_list:
#     #print(i[0])
# munge.sum_adjacent_days(met_df, adjacent_days_list, 'day', 'rain')
# # for i in adjacent_days_list:
# #     day1 = met_df.loc[met_df['day'] == i[0]]
# #     day1.head()
# #     #day1_precip = day1.iloc[0]['rain']
# # #top2 = munge.get_top2_precip_events(top10_precip_events_df, adjacent_days_list, 'day', precip_col)
# # #total_precip = munge.sum_met_precip(met_df, year, precip_col)
# # # ndvi_twi_gdf.insert(3, 'top_prec_2', top2[1])
# # # ndvi_twi_gdf.insert(3, 'top_prec_1', top2[0])
# # # ndvi_twi_gdf.insert(3, 'tot_precip', total_precip)
# # # ndvi_twi_met_gdf = ndvi_twi_gdf
# # # if ndvi_twi_met_gdf.empty == True:
# # #     print('ndvi_twi_met_gdf from prepare_met_df() is empty')
# # top10_precip_events_df
# # return ndvi_twi_met_gdf

In [ ]:
# def create_sampled_shapefile(field_list):
#     for i in field_list:
#         field = json.loads( open( f'field_processing_jsons/{i}.json', 'r' ).read() )
#         field_folder = f'out_files/{i}/shape_files'
#         # if not os.path.exists(field_folder):
#         #     os.mkdirs(field_folder)
#         for i in field['year']:
#             field_name = field['field_name']
#             twi_path = field['twi_file']
#             ssurgo_path = field['ssurgo_file']
#             ym_file = field['year'][i]['ym_file']
#             rotation = field['year'][i]['rotation']
#             crop = field['year'][i]['crop']
#             apsim_folder = field['apsim_files']
#             met_file = field['met_file']
#             june_ndvi_date = f'{i}06'
#             july_ndvi_date = f'{i}07'
#             june_ndvi_file = ''
#             july_ndvi_file = ''
#             #set target crs from gpd df
#             ym_gdf = gpd.read_file(ym_file)
#             target_crs = ym_gdf.crs
#             #search for june and july ndvi files
#             for file in os.listdir(f'ndvi/{field_name}'):
#                 if june_ndvi_date in file:
#                     june_ndvi_file = file
#                 if july_ndvi_date in file:
#                     july_ndvi_file = file
#             if june_ndvi_file == '':
#                 print(f'June NDVI file not found for {field_name} {i}.')
#                 break
#             if july_ndvi_file == '':
#                 print(f'July NDVI file not found for {field_name} {i}.')
#                 break
#             #sample twi to ym file
#             twi_out_path = f'out_files/{field_name}/{field_name}_twi_26915.tif'
#             twi_gdf = munge.prepare_twi_df(ym_file, twi_path, twi_out_path, target_crs, i, field_name, crop)
#             ## sample ndvi to new twi_gdf
#             #first get appropirate in and out paths
#             jun_ndvi_in_path = f'ndvi/{field_name}/{june_ndvi_file}'
#             jun_ndvi_out_path = f'out_files/{field_name}/{field_name}_ndvi_twi_jun_{i}_26915.tif'
#             jul_ndvi_in_path = f'ndvi/{field_name}/{july_ndvi_file}'
#             jul_ndvi_out_path = f'out_files/{field_name}/{field_name}_ndvi_twi_jul_{i}_26915.tif'
#             #then sample for june and for july
#             jun_ndvi_twi_gdf = munge.prepare_ndvi_df(twi_gdf, 'jun', jun_ndvi_in_path, jun_ndvi_out_path, target_crs, all_touched=False)
#             jul_ndvi_twi_gdf = munge.prepare_ndvi_df(jun_ndvi_twi_gdf, 'jul', jul_ndvi_in_path, jul_ndvi_out_path, target_crs, all_touched=False)
#             ## met
#             met_path = f'met_files/nasapwr/{met_file}'
#             ndvi_twi_met_gdf = munge.prepare_met_df(met_path, jul_ndvi_twi_gdf, i, header=7, precip_col='rain')
#             ## ssurgo
#             ssurgo_out_path = f'out_files/{field_name}/{field_name}_ssurgo_26915.geojson'
#             ndvi_twi_met_ssurgo_gdf = munge.prepare_ssurgo_df(ndvi_twi_met_gdf, ssurgo_path, ssurgo_out_path, target_crs)
#             print(ndvi_twi_met_ssurgo_gdf.head())
#             ## apsim files
#             apsim_files_path = f'apsim_files/{apsim_folder}/{i}/{rotation}'
#             write_file = f'{field_name}_{crop}_{i}_full'
#             munge.prepare_apsim_full_df(ndvi_twi_met_ssurgo_gdf, apsim_files_path, int(i), field_folder, write_file)